## 1. Analysis of costs to run animal shelters
<p>You are working as a data analyst in the animal welfare department. In preparation for next year's budget, the head of your department would like to know the total cost to shelter animals, broken down by each animal type and size combination (i.e., 'small dogs', 'large dogs', etc.). You need to prepare a table that contains this information.</p>
<p>The total cost to shelter an animal for the year is calculated as the sum of three factors: the size and animal type, the age, and the location.</p>
<ul>
<li>The base cost of sheltering an animal is based upon its size and type. The costs per animal type and size are contained in the <code>size_costs</code> table. The criteria for classifying size has recently been updated, and so you will need to categorize animals based upon the following table:</li>
</ul>
<table>
<thead>
<tr>
<th style="text-align:right;"></th>
<th style="text-align:right;">Small</th>
<th>Medium</th>
<th>Large</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:right;">Dog</td>
<td style="text-align:right;">&lt;= 10 lbs</td>
<td>10 lbs &lt; and &lt;= 30 lbs</td>
<td>30 lbs &lt;</td>
</tr>
<tr>
<td style="text-align:right;">Cat</td>
<td style="text-align:right;">&lt;= 5 lbs</td>
<td>5 lbs &lt; and &lt;= 7 lbs</td>
<td>7 lbs &lt;</td>
</tr>
<tr>
<td style="text-align:right;">Bird</td>
<td style="text-align:right;">&lt;= 0.7 lbs</td>
<td>0.7 lbs &lt; and &lt;= 1.1 lbs</td>
<td>1.1 lbs &lt;</td>
</tr>
</tbody>
</table>
<ul>
<li>Older animals cost more, and so an age cost (contained in the <code>age_costs</code> table) is added. Each animal's age should be calculated as the age by the end of the year (December 31st, 2021).</li>
<li>There is a location cost depending on where the animal is sheltered (contained in the <code>location_costs</code> table). </li>
<li>The calculation should not include animals that have been sponsored by private charities (sponsored animals are listed in the <code>sponsored_animals</code> table).</li>
</ul>
<p>For future visualization purposes, you will also need to include a <code>percentage</code> column in your result. This percentage should reflect the fraction of the total cost to be allocated to each animal and size combination.</p>
<p>The data you need is available in the tables shown in the database schema below.</p>
<h5 id="databaseschema">Database Schema</h5>
<p><img src="https://assets.datacamp.com/production/repositories/5934/datasets/a946a159c0024ee0995f7a030f2c0cf802203835/diagram.PNG" width="500" height="500"> </p>

In [32]:
%%sql
postgresql:///animal_shelters
Alter table animals DROP age;
ALTER table animals DROP size_id;

Done.
Done.


[]

In [33]:
%%sql
postgresql:///animal_shelters

ALTER TABLE animals ADD age int; 
ALTER TABLE animals ADD size_id varchar(50);
ALTER TABLE age_costs ALTER COLUMN age TYPE int USING age::integer; 

UPDATE animals 
SET Age = CAST(date_part('year', AGE(CAST('12-31-2021' as date), CAST(birthdate as date))) as int),
size_id = (CASE WHEN animaltype = 'Dog' 
           THEN (CASE WHEN weight<=10 THEN 'DS' WHEN weight>10 and weight<=30 THEN 'DM' WHEN weight>30 THEN 'DL' END)
           WHEN animaltype = 'Cat' 
           THEN (CASE WHEN weight<=5 THEN 'CS' WHEN weight>5 and weight<=7 THEN 'CM' WHEN weight>7 THEN 'CL' END)
           WHEN animaltype = 'Bird' 
           THEN (CASE WHEN weight<=0.7 THEN 'BS' WHEN weight>0.7 and weight<=1.1 THEN 'BM' WHEN weight>1.1 THEN 'BL' END)
          END)

           

Done.
Done.
Done.
11899 rows affected.


[]

In [34]:
%%sql
postgresql:///animal_shelters
    
select * 
from animals
limit 5;

5 rows affected.


animalid,birthdate,animaltype,color,weight,location,age,size_id
d3c29b4569fa66,01/18/2019,Dog,Black,39.0,San Antonio,2,DL
0a2749f68de7eb,02/05/2005,Dog,Black,17.0,Austin,16,DM
39bfe3d33a40dc,02/26/2014,Dog,Red,5.0,Dallas,7,DS
31c15d37470e7a,04/17/2018,Cat,Black,8.0,Houston,3,CL
2ca3a8c09c32fb,09/22/2015,Cat,Black,5.0,Houston,6,CS


In [35]:
%%sql
postgresql:///animal_shelters

CREATE TABLE result as (
select a.animaltype, sc.size, sum(ac.costs + sc.costs + lc.costs) as total
from animals as a  
left join sponsored_animals as sa on a.animalid = sa.sponsorid 
inner join age_costs as ac on a.age = ac.age
inner join size_costs as sc on a.size_id = sc.sizeid
inner join location_costs as lc on a.location = lc.location
where sa.sponsorid is null
group by a.animaltype, sc.size
order by a.animaltype, sc.size DESC
); 


(psycopg2.errors.DuplicateTable) relation "result" already exists

[SQL: CREATE TABLE result as (
select a.animaltype, sc.size, sum(ac.costs + sc.costs + lc.costs) as total
from animals as a  
left join sponsored_animals as sa on a.animalid = sa.sponsorid 
inner join age_costs as ac on a.age = ac.age
inner join size_costs as sc on a.size_id = sc.sizeid
inner join location_costs as lc on a.location = lc.location
where sa.sponsorid is null
group by a.animaltype, sc.size
order by a.animaltype, sc.size DESC
);]
(Background on this error at: http://sqlalche.me/e/14/f405)


In [36]:
%%sql
postgresql:///animal_shelters

select sum(total) into expense_total from result; 

(psycopg2.errors.DuplicateTable) relation "expense_total" already exists

[SQL: select sum(total) into expense_total from result;]
(Background on this error at: http://sqlalche.me/e/14/f405)


In [ ]:
%%sql
postgresql:///animal_shelters

select animaltype, size, total, round(((total/sum)*100),2) as percentage
from result, expense_total;